In [28]:
import pandas as pd
import numpy as np

In [29]:
from bs4 import BeautifulSoup             

In [31]:
import re

In [13]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier

In [34]:
from nltk.corpus import stopwords

In [40]:
# from KaggleWord2VecUtility import KaggleWord2VecUtility

-----
## Part 1: For Beginners - Bag of Words

### Reading the Data

In [32]:
train = pd.read_table('data/labeledTrainData.tsv', delimiter='\t', quoting=3)

train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [4]:
test = pd.read_table('data/testData.tsv', delimiter='\t', quoting=1)

test.head()

,id,review
0,12311_10,Naturally in a film who's main themes are of m...
1,8348_2,This movie is a disaster within a disaster fil...
2,5828_4,"All in all, this is a movie for kids. We saw i..."
3,7186_2,Afraid of the Dark left me with the impression...
4,12128_7,A very accurate depiction of small time mob li...


------
### Data Cleaning and Text Preprocessing

In [37]:
def review_to_words(raw_review):
    # Function to convert a raw review to a string of words
    # The input is a single string (a raw movie review), and 
    # the output is a single string (a preprocessed movie review)
    #
    # 1. Remove HTML
    review_text = BeautifulSoup(raw_review, 'lxml').get_text() 
    #
    # 2. Remove non-letters        
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    #
    # 3. Convert to lower case, split into individual words
    words = letters_only.lower().split()                             
    #
    # 4. In Python, searching a set is much faster than searching
    #   a list, so convert the stop words to a set
    stops = set(stopwords.words("english"))                  
    # 
    # 5. Remove stop words
    meaningful_words = [w for w in words if not w in stops]   
    #
    # 6. Join the words back into one string separated by space, 
    # and return the result.
    return( " ".join( meaningful_words ))   

# review_to_words( train["review"][0] )

In [39]:
clean_train_reviews = [review_to_words(review) for review in train.review]

------
### Creating Features from a Bag of Words (Using scikit-learn)

In [41]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000) 

In [ ]:
%%time
train_data_features = vectorizer.fit_transform(clean_train_reviews)